In [1]:
import os
import sys
import argparse
import cv2
import time
from config_reader import config_reader
from processing import extract_parts, draw

from model.cmu_model import get_testing_model

sys.path.append(os.path.join(os.path.dirname(os.path.abspath('')), ".."))

currentDT = time.localtime()
start_datetime = time.strftime("-%m-%d-%H-%M-%S", currentDT)

Using TensorFlow backend.
/home/ashish/Documents/UCD/InsightCentre/VirtualEnvironments/multits/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ashish/Documents/UCD/InsightCentre/VirtualEnvironments/multits/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ashish/Documents/UCD/InsightCentre/VirtualEnvironments/multits/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood

In [6]:
keras_weights_file = "model.h5"
frame_rate_ratio = 1
process_speed = 4
ending_frame = None

In [21]:
# Video input
video = "/home/ashish/Downloads/HumanPose/Binh_VideoDataset_June2019/TimeSeries_Exp4/test.mp4"
video_path = 'videos/'
video_file = video

# Output location
output_path = 'videos/outputs/'
output_format = '.mp4'
video_output = output_path + video + str(start_datetime) + output_format

In [22]:
# vgg normalization (subtracting mean) on input images
model = get_testing_model()
model.load_weights(keras_weights_file)

In [14]:
params, model_params = config_reader()


In [15]:
params

{'use_gpu': 1, 'GPUdeviceNumber': 0, 'modelID': '1', 'octave': 3, 'starting_range': 0.8, 'ending_range': 2.0, 'scale_search': [0.5, 1.0, 1.5, 2.0], 'thre1': 0.1, 'thre2': 0.05, 'thre3': 0.5, 'min_num': 4, 'mid_num': 10, 'crop_ratio': 2.5, 'bbox_ratio': 0.25}

In [11]:
model_params

{'caffemodel': './model/_trained_COCO/pose_iter_440000.caffemodel', 'deployFile': './model/_trained_COCO/pose_deploy.prototxt', 'description': 'COCO Pose56 Two-level Linevec', 'boxsize': 368, 'padValue': 128, 'np': '12', 'stride': 8, 'part_str': ['[nose', 'neck', 'Rsho', 'Relb', 'Rwri', 'Lsho', 'Lelb', 'Lwri', 'Rhip', 'Rkne', 'Rank', 'Lhip', 'Lkne', 'Lank', 'Leye', 'Reye', 'Lear', 'Rear', 'pt19]']}

In [29]:

# Video reader
cam = cv2.VideoCapture(video_file)
input_fps = cam.get(cv2.CAP_PROP_FPS)
ret_val, orig_image = cam.read()
video_length = int(cam.get(cv2.CAP_PROP_FRAME_COUNT))

In [23]:
def manual_count(handler):
    frames = 0
    while True:
        status, frame = handler.read()
        if not status:
            break
        frames += 1
    return frames 

In [24]:
cam = cv2.VideoCapture(video_file)

In [25]:
manual_count(cam)

241

In [30]:
if ending_frame is None:
    ending_frame = video_length

In [31]:
frame_rate_ratio

1

In [32]:
input_fps

29.97002997002997

In [21]:
output_fps = input_fps / frame_rate_ratio
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(video_output, fourcc, output_fps, (orig_image.shape[1], orig_image.shape[0]))

scale_search = [1, .5, 1.5, 2]  # [.5, 1, 1.5, 2]
scale_search = scale_search[0:process_speed]
params['scale_search'] = scale_search


In [26]:
output_fps

29.97002997002997

In [27]:
i = 0  # default is 0
while(cam.isOpened()) and ret_val is True and i < ending_frame:
    if i % frame_rate_ratio == 0:

        input_image = cv2.cvtColor(orig_image, cv2.COLOR_RGB2BGR)

        tic = time.time()

        # generate image with body parts
        body_parts, all_peaks, subset, candidate = extract_parts(input_image, params, model, model_params)
        # canvas = draw(orig_image, all_peaks, subset, candidate)

        print('Processing frame: ', i)
        toc = time.time()
        print('processing time is %.5f' % (toc - tic))

        # out.write(canvas)

    ret_val, orig_image = cam.read()

    i += 1


Processing frame:  0
processing time is 22.05920


In [28]:
body_parts

{'nose': (483, 167),
 'neck': (483, 203),
 'right_shoulder': (455, 204),
 ' right_elbow': (446, 244),
 'right_wrist': (446, 285),
 'left_shoulder': (510, 203),
 'left_elbow': (522, 244),
 'left_wrist': (528, 283),
 'right_hip': (469, 284),
 'right_knee': (463, 352),
 'right_ankle': (455, 421),
 'left_hip': (505, 283),
 'left_knee': (509, 350),
 'left_ankle': (516, 420),
 'right_eye': (478, 162),
 'left_eye': (488, 162),
 'right_ear': (471, 168),
 'left_ear': (496, 168)}

In [29]:
all_peaks

[[(483, 167, 0.9262133538722992, 0)],
 [(483, 203, 0.934542253613472, 1)],
 [(455, 204, 0.899353638291359, 2)],
 [(446, 244, 0.8779154121875763, 3)],
 [(446, 285, 0.8312231004238129, 4)],
 [(510, 203, 0.8871498703956604, 5)],
 [(522, 244, 0.8205071836709976, 6)],
 [(528, 283, 0.7493290826678276, 7)],
 [(469, 284, 0.7184247523546219, 8)],
 [(463, 352, 0.8270892202854156, 9)],
 [(455, 421, 0.8078549355268478, 10)],
 [(505, 283, 0.7286079674959183, 11)],
 [(509, 350, 0.8157999813556671, 12)],
 [(516, 420, 0.7970129102468491, 13)],
 [(478, 162, 0.9284635633230209, 14)],
 [(488, 162, 0.9314068257808685, 15)],
 [(471, 168, 0.841595932841301, 16)],
 [(496, 168, 0.874936044216156, 17)]]

In [30]:
subset

array([[ 0.        ,  1.        ,  2.        ,  3.        ,  4.        ,
         5.        ,  6.        ,  7.        ,  8.        ,  9.        ,
        10.        , 11.        , 12.        , 13.        , 14.        ,
        15.        , 16.        , 17.        , 31.90221745, 18.        ]])

In [31]:
candidate

array([[483.        , 167.        ,   0.92621335,   0.        ],
       [483.        , 203.        ,   0.93454225,   1.        ],
       [455.        , 204.        ,   0.89935364,   2.        ],
       [446.        , 244.        ,   0.87791541,   3.        ],
       [446.        , 285.        ,   0.8312231 ,   4.        ],
       [510.        , 203.        ,   0.88714987,   5.        ],
       [522.        , 244.        ,   0.82050718,   6.        ],
       [528.        , 283.        ,   0.74932908,   7.        ],
       [469.        , 284.        ,   0.71842475,   8.        ],
       [463.        , 352.        ,   0.82708922,   9.        ],
       [455.        , 421.        ,   0.80785494,  10.        ],
       [505.        , 283.        ,   0.72860797,  11.        ],
       [509.        , 350.        ,   0.81579998,  12.        ],
       [516.        , 420.        ,   0.79701291,  13.        ],
       [478.        , 162.        ,   0.92846356,  14.        ],
       [488.        , 162

In [ ]:
 body_parts, all_peaks, subset, candidate

In [32]:
import numpy as np
import cv2

In [33]:
import os
os.getcwd()

'/home/ashish/Downloads/keras_Realtime_Multi-Person_Pose_Estimation-master'

In [34]:
!ls

caffe_to_keras.py  demo.ipynb		 processing.py	util.py
config		   demo_video.py	 __pycache__	video_coordinates.ipynb
config_reader.py   dump_caffe_layers.py  readme		videos
dataset		   LICENSE		 README.md
demo_camera.py	   model		 sample_images
demo_image.py	   model.h5		 training


In [35]:
cap = cv2.VideoCapture("./videos/sample1.mp4")
while(cap.isOpened()):
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow('frame', gray)
    if cv2.waitKey(1) & 0xff == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.1.2) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
